## Part 5: Testing our neural network on our own handwritten digits

In this last tutorial, we'll load an image of a handwritten digit that I took and test our neural network on it. Being that my handwriting is not a part of MNIST's dataset, I don't expect the neural network to always know what my digits are. 

At the end of this tutorial, we'll provide graph paper for you to write your own digits on, then we'll test the neural network on a bunch of handwritten digits, instead of just a few. We'll see how well our NN will be able to interpret handwriting that isn't similar to its training set.

Just like before, we'll load our NN and our training/testing set, as well as the path to the image of my handwritten 5.

In [ ]:
# import libraries and some functions that are written for you 
from handwrite_functions import *
model = torch.load('./my_mnist_model.pt')
# where you put your image of a digit
load_mydigit = "../datasets/five_real.png"

In [ ]:
# where you want to save your dataset
dataset = 'MY_DATASET'
testset = 'MY_TESTSET'

transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# download training sets and test sets 
trainset = datasets.MNIST(dataset, download=False, train=True, transform=transform)
valset = datasets.MNIST(testset, download=False, train=False, transform=transform)

# load training sets and test sets in batch sizes of 64
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)

# prepare loaded data sets to iterate over
dataiter = iter(trainloader)
images, labels = next(dataiter)

We'll load our custom image and take a look at the size/shape of the pixels. (What are pixels and what do the numbers mean?)

In [ ]:
# load your custom image of a digit
custom_image_path = load_mydigit

# Read in custom image
custom_image_uint8 = torchvision.io.read_image(str(custom_image_path))

# Print out image data to check if reading worked.
print("These are the shading (0 - 255) of each pixel")
print(f"Custom image tensor:\n{custom_image_uint8}\n")
print(f"Custom image shape: {custom_image_uint8.shape}\n")
print(f"Custom image dtype: {custom_image_uint8.dtype}")

Maybe our image has some noise and we want to apply a filter. You can try this step and load your de-noised image to test it in the next part. De-noising looks like applying a smoothing filter!

In [ ]:
# want to "de-noise" your image? Try this after naming your "output_image_path"
output_image_path = "/Users/sierra/Downloads/five_denoise.png"
denoise_img(custom_image_path, output_image_path)

---
Now, we'll normalize the pixels in the image to make this compatible with our neural network. We should only see 0s and 1s in the output. 

In [ ]:
# Load in your custom image and convert the values to floats 
custom_image = torchvision.io.read_image(str(custom_image_path)).type(torch.float32)

# Divide each pixel value by 255 to get them between [0, 1]
custom_image = custom_image / 255. 

# Invert the colors so that the digit is black on a light background
custom_image = np.abs(custom_image - 1)

# Rount each pixel to 1 or 0 for maximum contrast!
custom_image = np.round(custom_image)
# Print out image data - does it look like the other one?
print(f"Custom image tensor:\n{custom_image}\n")
print(f"Custom image shape: {custom_image.shape}\n")
print(f"Custom image dtype: {custom_image.dtype}")

If we try to test our neural network on this, it won't know what to do! The MNIST dataset contains images with 28x28 pixels. The image I took has 924x756 pixels, so the NN will not be able to operate on the mismatched sizes. 

Luckily, PyTorch has a function to resize our image into 28x28. Whew!

In [ ]:
# We need to make sure that the original image matches in shape, otherwise our NN will be confused
custom_image_transform = transforms.Compose([
    transforms.Resize((28, 28)), #MNIST only provides 28x28 so let's change ours to match
])

# Transform target image
custom_image_transformed = custom_image_transform(custom_image)

# Print out original shape and new shape
print(f"Original shape: {custom_image.shape}")
print(f"New shape: {custom_image_transformed.shape}")

It's finally time to test our image. 

In [ ]:
# load our image!
img = custom_image_transformed[0].view(1, 784)

# plot our image as seen by torchvision
plt.imshow(custom_image_transformed[0].numpy().squeeze(), cmap='gray_r')
plt.axis(False);

# use our NN to test on our image
with torch.no_grad():
    logps = model(img)

ps = torch.exp(logps)
probab = list(ps.numpy()[0])
print("Predicted Digit =", probab.index(max(probab)))
print(f"How sure?: {max(probab) * 100:.4} %")

Since my NN wasn't super confident (although it got the right answer!), I want to know what else the neural network thought it could be. Mine suggests that it could be a `3` or a `9`. Do you see how it could be mistaken?

In [ ]:
# Just for fun, let's look at what the NN "thought" of other options.
for i in np.array(probab):
    print(f"{i * 100:.4}% digit could be {probab.index(i)}")
    
# Are there any that come close? Do you see why the NN could "think" it would be another digit?

# Activity:
1. We'll provide graph paper and have you fill in digits 0-9 a few times. You'll scan this and save it to a file. There's a script to split the image into test images with labels, so we'll try to test this neural network on that. Show us the steps to read in those images and save an image of the confusion matrix that results! (This is quite a few steps, so work with each other and ask for help if needed).



---
Congratulations on finishing these tutorials! Next, we'll try to do the whole process again on neutrino interactions. 